In [ ]:
import scipy.io
import matplotlib.pyplot as plt
import os
import glob
from scipy.stats import entropy
import sys
import matplotlib.pyplot as plt
import pywt
import numpy as np
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import pandas as pd
import numpy as np
from scipy.signal import welch
from scipy.interpolate import interp1d

In [ ]:
"""Mounting Google Drive"""
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def find_mat_files(root_dir):
    mat_files = []
    for dirpath, _, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.endswith('.mat'):
                mat_files.append(os.path.join(dirpath, filename))
    return mat_files

# file_path = r'/content/drive/MyDrive/ASD/'
file_path = r'/content/drive/MyDrive/TD/'

mat_files = find_mat_files(file_path)

In [ ]:
# Function to perform window slicing on the EEG signal
def window_slice(signal, window_size, overlap):
    """
    Perform window slicing on the signal.

    Parameters:
    - signal: The input signal (1D array).
    - window_size: The size of each window.
    - overlap: The overlap between consecutive windows.

    Returns:
    - windows: 2D array containing sliced windows.
    """
    windows = []
    for i in range(0, len(signal) - window_size + 1, window_size - overlap):
        window = signal[i:i + window_size]
        windows.append(window)
    return np.array(windows)

In [ ]:
# Function to extract time-domain features from each window
def extract_time_domain_features(window):
    features = [
        np.mean(window),
        np.std(window),
        np.median(window),
        np.max(window) - np.min(window),
        np.mean((window - np.mean(window)) ** 3) / np.std(window) ** 3,  # Skewness
        np.mean((window - np.mean(window)) ** 4) / np.var(window) ** 2 - 3,  # Kurtosis
    ]
    return features

In [ ]:
# Function to extract frequency band power from each window
def extract_frequency_bands(window, sampling_rate):
    f, psd = welch(window, fs=sampling_rate, nperseg=len(window))

    # Define frequency bands (adjust as needed)
    delta_band = (0.5, 4)
    theta_band = (4, 8)
    alpha_band = (8, 13)
    beta_band = (13, 30)
    gamma_band = (30, 40)

    # Interpolate PSD to ensure even frequency spacing
    f_interp = np.linspace(f.min(), f.max(), len(f))
    psd_interp = interp1d(f, psd, kind='linear', fill_value='extrapolate')(f_interp)

    # Calculate index range for each frequency band
    delta_indices = np.where((f_interp >= delta_band[0]) & (f_interp <= delta_band[1]))[0]
    theta_indices = np.where((f_interp >= theta_band[0]) & (f_interp <= theta_band[1]))[0]
    alpha_indices = np.where((f_interp >= alpha_band[0]) & (f_interp <= alpha_band[1]))[0]
    beta_indices = np.where((f_interp >= beta_band[0]) & (f_interp <= beta_band[1]))[0]
    gamma_indices = np.where((f_interp >= gamma_band[0]) & (f_interp <= gamma_band[1]))[0]

    # Extract power in each frequency band
    delta_power = np.trapz(psd_interp[delta_indices], dx=f_interp[1] - f_interp[0])
    theta_power = np.trapz(psd_interp[theta_indices], dx=f_interp[1] - f_interp[0])
    alpha_power = np.trapz(psd_interp[alpha_indices], dx=f_interp[1] - f_interp[0])
    beta_power = np.trapz(psd_interp[beta_indices], dx=f_interp[1] - f_interp[0])
    gamma_power = np.trapz(psd_interp[gamma_indices], dx=f_interp[1] - f_interp[0])

    return {
        'Delta': delta_power,
        'Theta': theta_power,
        'Alpha': alpha_power,
        'Beta': beta_power,
        'Gamma': gamma_power
    }

In [ ]:
mat_file_path
mat_data = scipy.io.loadmat(mat_file_path)
mat_data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'Fix_Segment1', 'Fix_Segment2', 'Fix_Segment3', 'Fix_Segment4', 'Fix_Segment5', 'Fix_Segment6', 'Fix_Segment7', 'Fix_Segment8', 'Fix_Segment9', 'Fix_Segment10', 'Fix_Segment11', 'Fix_Segment12', 'Fix_Segment13', 'Fix_Segment14', 'Fix_Segment15', 'Fix_Segment16', 'Fix_Segment17', 'Fix_Segment18', 'Fix_Segment19', 'Fix_Segment20', 'Fix_Segment21', 'Fix_Segment22', 'Fix_Segment23', 'Fix_Segment24', 'Fix_Segment25', 'Fix_Segment26', 'Fix_Segment27', 'Fix_Segment28', 'Fix_Segment29', 'Fix_Segment30', 'Fix_Segment31', 'Fix_Segment32', 'Fix_Segment33', 'Fix_Segment34', 'Fix_Segment35', 'Fix_Segment36', 'Fix_Segment37', 'Fix_Segment38', 'Fix_Segment39', 'Fix_Segment40', 'Fix_Segment41', 'Fix_Segment42', 'Fix_Segment43', 'Fix_Segment44', 'Fix_Segment45', 'Fix_Segment46', 'Fix_Segment47', 'Fix_Segment48', 'Fix_Segment49', 'Fix_Segment50', 'Fix_Segment51', 'Fix_Segment52', 'Fix_Segment53', 'Fix_Segment54', 'Fix_Segment55', 'Fix_Segment56', 'F

In [ ]:
for mat_file_path in mat_files:
  mat_data = scipy.io.loadmat(mat_file_path)
  try:
    eeg_data = mat_data['Bio_Segment4']
  except:
    try:
      eeg_data = mat_data['Known_Segment4']
    except:
      eeg_data = mat_data['EO_Segment4']

  print("shape of bio segment 1",eeg_data.shape)

  if len(mat_files) == 1:

    # Plot EEG channels
    num_channels, num_time_points = eeg_data.shape

    for channel in range(num_channels):
        plt.plot(eeg_data[channel, :], label=f'Channel {channel + 1}')

    plt.title('EEG Data Plot')
    plt.xlabel('Time Points')
    plt.ylabel('Amplitude')
    plt.legend()
    plt.show()

    # Plot the top 5 EEG channels
    num_channels, num_time_points = eeg_data.shape
    num_channels_to_plot = min(5, num_channels)  # Choose the top 5 channels or all channels if less than 5

    for channel in range(num_channels_to_plot):
        plt.plot(eeg_data[channel, :], label=f'Channel {channel + 1}')

    plt.title('Top 5 EEG Channels')
    plt.xlabel('Time Points')
    plt.ylabel('Amplitude')
    plt.legend()
    plt.show()


    w = pywt.Wavelet('db4')
    maxlev = pywt.dwt_max_level(len(eeg_data), w.dec_len)
    print("maximum level is " + str(maxlev))
    threshold = 0.04
    coeffs = pywt.wavedec(eeg_data, 'db4', level=3)
    # Extract the approximation coefficients at the chosen level
    cA = coeffs[0]

  if len(mat_files) == 1:
    # Plot the approximation coefficients
    plt.plot(cA, '--*b')
    plt.title(f'Approximation Coefficients at Level {maxlev}')
    plt.xlabel('Sample Index')
    plt.ylabel('Coefficient Value')
    plt.show()

  # Threshold the wavelet coefficients
  for i in range(1, len(coeffs)):  # Skip the approximation coefficients
      coeffs[i] = pywt.threshold(coeffs[i], threshold * np.max(np.abs(coeffs[i])), mode='soft')

  # Reconstruct the signal using the thresholded coefficients
  reconstructed_signal = pywt.waverec(coeffs, 'db4')
  print(reconstructed_signal.shape)

  if len(mat_files) == 1:
    # Plot the original and reconstructed signals
    plt.figure(figsize=(10, 6))
    plt.plot(eeg_data, label='Original Signal', alpha=0.7)
    plt.plot(reconstructed_signal, label='Reconstructed Signal', linestyle='--', linewidth=2)
    plt.title('Original and Reconstructed Signals after Wavelet Thresholding')
    plt.xlabel('Sample Index')
    plt.ylabel('Signal Value')
    plt.legend()
    plt.show()


  # Parameters for window slicing
  window_size = 100  # Size of each window
  overlap = 50  # Overlap between consecutive windows


  # Apply window slicing to the EEG signal
  eeg_windows = window_slice(reconstructed_signal, window_size, overlap)
  eeg_windows = np.reshape(eeg_windows, (100, 600))

  if len(mat_file_path) == 1:

    # Plot the original EEG signal and the sliced windows
    plt.figure(figsize=(10, 6))

    plt.subplot(2, 1, 1)
    plt.plot(reconstructed_signal)
    plt.title('Reconstructed EEG Signal')
    plt.xlabel('Sample')
    plt.ylabel('Amplitude')

    plt.subplot(2, 1, 2)

    for i, window in enumerate(eeg_windows):
        plt.subplot(len(eeg_windows), 1, i + 1)
        plt.plot(window, color='b')
        plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))
        plt.title(f'Window {i + 1}')
        plt.xlabel('Sample')
        plt.ylabel('Amplitude')

    # Adjust layout and save to an Excel file
    plt.tight_layout()
    plt.savefig('eeg_windows_16.png')
    plt.close()


  # Extract time-domain features for each window
  time_domain_features = [extract_time_domain_features(window) for window in eeg_windows]

  # Convert the list of features to a NumPy array
  time_domain_features_array = np.array(time_domain_features)

  # Print or use the features as needed
  print("Time-domain features shape:", time_domain_features_array.shape)
  print("Example features for the first window:", time_domain_features_array[0])


  # Save features to Excel file
  feature_names = ['Mean', 'Std', 'Median', 'Range', 'Skewness', 'Kurtosis']
  df = pd.DataFrame(time_domain_features_array, columns=feature_names)

  # Save the DataFrame to an Excel file
  folder_name = os.path.basename(os.path.dirname(mat_file_path))
  folder_number = ''.join(filter(str.isdigit, folder_name))

  # Create Excel file name
  # excel_file_path = '/content/drive/MyDrive/Features ASD_TD/ASD/' + folder_number + '.xlsx'
  excel_file_path = '/content/drive/MyDrive/Features ASD_TD/TD/' + folder_number + '.xlsx'

  df.to_excel(excel_file_path, index=False)

  print(f'Time-domain features saved to {excel_file_path}')


  # Sample rate of the EEG signal (adjust as needed)
  sampling_rate = 1000  # Hz

  # Extract frequency band power for each window
  frequency_band_powers = [extract_frequency_bands(window, sampling_rate) for window in eeg_windows]

  # Convert the list of features to a pandas DataFrame
  frequency_band_df = pd.DataFrame(frequency_band_powers)
  # Save the DataFrame to an Excel file
  # excel_file_path = '/content/drive/MyDrive/Features ASD_TD/ASD/' + folder_number + '_bands.xlsx'
  excel_file_path = '/content/drive/MyDrive/Features ASD_TD/TD/' + folder_number + '_bands.xlsx'

  frequency_band_df.to_excel(excel_file_path, index=False)
  print(f'Frequency band powers saved to {excel_file_path}')

shape of bio segment 1 (125, 512)
(125, 600)
Time-domain features shape: (100, 6)
Example features for the first window: [25.31581159  7.88065481 25.53654805 34.44876576  0.09168136 -0.80195737]
Time-domain features saved to /content/drive/MyDrive/Features ASD_TD/TD/1.xlsx
Frequency band powers saved to /content/drive/MyDrive/Features ASD_TD/TD/1_bands.xlsx
shape of bio segment 1 (125, 600)
(125, 600)
Time-domain features shape: (100, 6)
Example features for the first window: [25.31581159  7.88065481 25.53654805 34.44876576  0.09168136 -0.80195737]
Time-domain features saved to /content/drive/MyDrive/Features ASD_TD/TD/3.xlsx
Frequency band powers saved to /content/drive/MyDrive/Features ASD_TD/TD/3_bands.xlsx
shape of bio segment 1 (125, 512)
(125, 600)
Time-domain features shape: (100, 6)
Example features for the first window: [25.31581159  7.88065481 25.53654805 34.44876576  0.09168136 -0.80195737]
Time-domain features saved to /content/drive/MyDrive/Features ASD_TD/TD/7.xlsx
Freque